# **Praktikum 3**

In [ ]:
!pip install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2527738 sha256=4f0dec1faf817d86dd86e14c93cbded2ac3f1e3ab71185501735def0b7a322c7
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built hnswlib


In [ ]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[234 938 622 789 400]]
Distances: [[0.01163715 0.03473121 0.03580073 0.04402807 0.04525989]]
Waktu: 0.0037152767181396484 detik

=== HNSW ===
Indices: [[234 938 622 789 400]]
Distances: [[0.00013542 0.00120626 0.00128169 0.00193847 0.00204846]]
Waktu: 0.00019979476928710938 detik


Lakukan percobaan pada metric distance yang berbeda, 1000 vs 1jt data, 2D vs 5D data. catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.

In [ ]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors
import pandas as pd

def calculate_recall(ground_truth, approximate_results, k):
    """Menghitung recall@k."""
    correct = 0
    total = len(ground_truth) * k
    for i in range(len(ground_truth)):
        truth_set = set(ground_truth[i])
        approx_set = set(approximate_results[i])
        correct += len(truth_set.intersection(approx_set))
    return (correct / total) * 100

def run_experiment(dim, num_elements, space):
    """Menjalankan satu skenario eksperimen."""

    # 1. Tentukan parameter
    k = 5  # Jumlah tetangga terdekat dari kode Anda
    num_queries = 100 # Gunakan 100 query untuk hasil waktu yang lebih stabil

    # HNSW Parameters
    ef_construction = 100
    M = 16
    ef_search = 50

    # 2. Buat data
    np.random.seed(42)
    data = np.random.random((num_elements, dim)).astype(np.float32)
    queries = np.random.random((num_queries, dim)).astype(np.float32)

    # Map metric HNSW ke Sklearn
    if space == 'l2':
        sklearn_metric = 'euclidean'
    elif space == 'ip':
        # 'ip' (Inner Product) di HNSW sebanding dengan 'cosine' di sklearn
        # JIKA datanya dinormalisasi L2. Mari kita normalisasi.
        sklearn_metric = 'cosine'

        # Normalisasi data untuk perbandingan 'ip' vs 'cosine'
        norm_data = np.linalg.norm(data, axis=1, keepdims=True)
        norm_queries = np.linalg.norm(queries, axis=1, keepdims=True)
        # Hindari pembagian dengan nol jika ada vektor nol
        norm_data[norm_data == 0] = 1
        norm_queries[norm_queries == 0] = 1

        data = data / norm_data
        queries = queries / norm_queries
    else:
        raise ValueError("Metric tidak didukung")

    # --- 3. Percobaan Exact NN (Sklearn) ---
    nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=sklearn_metric)

    # Waktu Build (Fit)
    start_build_exact = time.time()
    nn.fit(data)
    time_build_exact = time.time() - start_build_exact

    # Waktu Search
    start_search_exact = time.time()
    distances_exact, indices_exact = nn.kneighbors(queries)
    time_search_exact = time.time() - start_search_exact

    # --- 4. Percobaan Approximate NN (HNSW) ---
    p = hnswlib.Index(space=space, dim=dim)

    # Waktu Build (init + add)
    start_build_hnsw = time.time()
    p.init_index(max_elements=num_elements, ef_construction=ef_construction, M=M)
    p.add_items(data)
    time_build_hnsw = time.time() - start_build_hnsw

    # Waktu Search
    p.set_ef(ef_search)
    start_search_hnsw = time.time()
    indices_hnsw, distances_hnsw = p.knn_query(queries, k=k)
    time_search_hnsw = time.time() - start_search_hnsw

    # --- 5. Hitung Recall ---
    recall = calculate_recall(indices_exact, indices_hnsw, k)

    return {
        'Metric': space,
        'Data': num_elements,
        'Dim (d)': dim,
        'Params (M, efC)': f"({M}, {ef_construction})",
        'Time Build (Exact)': f"{time_build_exact:.6f} s",
        'Time Search (Exact)': f"{time_search_exact:.6f} s",
        'Time Build (HNSW)': f"{time_build_hnsw:.6f} s",
        'Time Search (HNSW)': f"{time_search_hnsw:.6f} s",
        'Speedup (Search)': f"{time_search_exact / time_search_hnsw:.2f}x",
        'Recall @5': f"{recall:.2f}%"
    }

# --- Daftar Semua Eksperimen ---
experiments = [
    # Metrik L2 (Euclidean)
    {'dim': 2, 'num_elements': 1000, 'space': 'l2'},
    {'dim': 5, 'num_elements': 1000, 'space': 'l2'},
    {'dim': 2, 'num_elements': 1000000, 'space': 'l2'},
    {'dim': 5, 'num_elements': 1000000, 'space': 'l2'},
    # Metrik IP (Inner Product)
    {'dim': 2, 'num_elements': 1000, 'space': 'ip'},
    {'dim': 5, 'num_elements': 1000, 'space': 'ip'},
    {'dim': 2, 'num_elements': 1000000, 'space': 'ip'},
    {'dim': 5, 'num_elements': 1000000, 'space': 'ip'},
]

results = []
print("Memulai Eksperimen HNSW... (Eksperimen 1 Juta data akan memakan waktu)")

for i, params in enumerate(experiments):
    print(f"Menjalankan eksperimen {i+1}/{len(experiments)}: {params}")
    results.append(run_experiment(**params))

print("\nEksperimen Selesai.")

# --- Tampilkan Hasil dalam Tabel ---
df_results = pd.DataFrame(results)
# Atur kolom agar lebih mudah dibaca
columns_order = [
    'Metric', 'Data', 'Dim (d)', 'Params (M, efC)',
    'Time Build (Exact)', 'Time Build (HNSW)',
    'Time Search (Exact)', 'Time Search (HNSW)',
    'Speedup (Search)', 'Recall @5'
]
df_results = df_results[columns_order]

print(df_results.to_string())

Memulai Eksperimen HNSW... (Eksperimen 1 Juta data akan memakan waktu)
Menjalankan eksperimen 1/8: {'dim': 2, 'num_elements': 1000, 'space': 'l2'}
Menjalankan eksperimen 2/8: {'dim': 5, 'num_elements': 1000, 'space': 'l2'}
Menjalankan eksperimen 3/8: {'dim': 2, 'num_elements': 1000000, 'space': 'l2'}
Menjalankan eksperimen 4/8: {'dim': 5, 'num_elements': 1000000, 'space': 'l2'}
Menjalankan eksperimen 5/8: {'dim': 2, 'num_elements': 1000, 'space': 'ip'}
Menjalankan eksperimen 6/8: {'dim': 5, 'num_elements': 1000, 'space': 'ip'}
Menjalankan eksperimen 7/8: {'dim': 2, 'num_elements': 1000000, 'space': 'ip'}
Menjalankan eksperimen 8/8: {'dim': 5, 'num_elements': 1000000, 'space': 'ip'}

Eksperimen Selesai.
  Metric     Data  Dim (d) Params (M, efC) Time Build (Exact) Time Build (HNSW) Time Search (Exact) Time Search (HNSW) Speedup (Search) Recall @5
0     l2     1000        2       (16, 100)         0.000903 s        0.025601 s          0.002042 s         0.001121 s            1.82x   100.